In [1]:
# # load env variables, NOTE: this cell must be run first
# from dotenv import load_dotenv
# load_dotenv(override=True)
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time

In [2]:
os.environ['TRANSFORMERS_OFFLINE']="1"

In [3]:
os.environ['TRANSFORMERS_CACHE'] = '/scratch/shareddata/dldata/huggingface-hub-cache/hub'

In [4]:
import torch
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "gpt2"

In [12]:
# policy network
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name,
                                                         load_in_8bit=True,
                                                  torch_dtype=torch.float16,
                                                 device_map="cuda:0"
                                                         )
model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(model_name,
                                                         load_in_8bit=True,
                                                  torch_dtype=torch.float16,
                                                 device_map="cuda:1"
                                                         )

Loading checkpoint shards: 100%|██████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.22s/it]


In [8]:
def print_trainable_parameters(model):
    """
    Print the names and shapes of trainable parameters in a Hugging Face model.

    Args:
    model: A Hugging Face model instance.
    """
    trainable_params = 0
    all_params = 0
    for name, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable_params: {trainable_params}")
    print(f"all_params: {all_params}")
    
print_trainable_parameters(model)

trainable_params: 262414337
all_params: 6738419713


In [13]:

### Print out the layers, data types, and devices


# for name, module in model.named_modules():
#     for param_name, param in module.named_parameters(recurse=False):
#         print(f"Layer: {name}, Parameter: {param_name}, Type: {param.dtype}, Device: {param.device}")

In [14]:
tokenizer = tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [15]:
model

AutoModelForCausalLMWithValueHead(
  (pretrained_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_

In [16]:
ppo_config = {"batch_size":1}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config,model,model_ref,tokenizer)

In [17]:
model.pretrained_model.device

device(type='cuda', index=0)

In [26]:
query = "Are you happy"  # Define your query

inputs = tokenizer.encode(query, return_tensors="pt").to(model.pretrained_model.device)  


In [29]:
generation_kwargs = {"min_length": -1, 
                     "top_k": 0.0, 
                     "top_p": 1.0, 
                     "do_sample": True, 
                     "pad_token_id": tokenizer.eos_token_id
                    }
outputs = ppo_trainer.generate([item for item in inputs],return_prompt= True, max_length=50)
respond = tokenizer.decode(outputs[0], skip_special_tokens=True)
respond

'Are you happy with your current salary?\nDo you feel like you are being paid what you are worth?\nI am sure you have heard the saying, “If you are not getting paid what you are worth, you are worth what'

In [33]:
prompts = ['How do you like my new hair cut?',
           'Do you like Tayler Swift?'
          ]

In [37]:

inputs = tokenizer(prompts, padding=True,
                   truncation=True,
                   max_length=30, 
                   return_tensors="pt"
                  ).to(model.pretrained_model.device)

outputs = model.generate(**inputs, max_length=50)
    
    # Decode the outputs
responds = tokenizer.batch_decode(outputs, skip_special_tokens=True)


responds

['How do you like my new hair cut?\nI got it from the barber shop in the mall.\nThe one with the big mirrors.\nI like it, but I think you should wear it with a hat.\n',
 "Do you like Tayler Swift?\nI don't like her but I like her songs.\nI don't like her but I like her songs. I like her songs, but I don't like her.\n"]

In [38]:
# Use a pipeline as a high-level helper
# Use a pipeline as a high-level helper
from transformers import pipeline

classifier = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis")

In [39]:
results = classifier(responds)
results

[{'label': 'POS', 'score': 0.5353606343269348},
 {'label': 'NEU', 'score': 0.838121235370636}]

In [40]:
rewards = []
for result in results:
    if result['label']=='POS':
        reward = result['score']
    else: reward = -result['score']
    rewards.append(reward)

rewards

[0.5353606343269348, -0.838121235370636]

In [42]:
input_tensors = tokenizer(prompts, padding=True, truncation=True,\
                   max_length=30, return_tensors="pt")['input_ids'].to(model.pretrained_model.device)
input_tensors = []
for prompt in prompts:
    input_ = tokenizer(prompt, padding=True, truncation=True,\
                   max_length=30, return_tensors="pt")['input_ids']
    input_tensors.append(input_.squeeze())

In [ ]:
#### Get response 
response_tensors = []
for input_tensor in input_tensors:
    response = ppo_trainer.generate(input_tensor, **generation_kwargs)
    response_tensors.append(response.squeeze())
response_tensors

In [ ]:
return response
    outputs = model.generate(**inputs)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response



# Example usage
query = "Are you happy"  # Define your query
response = generate_response(query)
reward = get_reward(response)

# Step the trainer
ppo_trainer.step(reward)

# Save the model
model.save_pretrained("./trained_model")

In [ ]:
def zero_shot_classify_sequences(examples, threshold=0.5):

    # first, send batch of texts through pipeline
    texts = examples['text']
    outputs = classifier(texts, candidate_labels, multi_label=True)
    # next, for each output:
    final_outputs = []
    for output in outputs:
        # create dictionary (predicted_labels, confidence)
        final_output = {}
        for label, score in zip(output['labels'], output['scores']):
            if score > threshold:
               final_output[label] = score
        final_outputs.append(final_output)

    assert len(final_outputs) == len(texts)
    # set final outputs
    examples['predicted_labels'] = final_outputs

    return examples

In [ ]:
updated_dataset = dataset.map(zero_shot_classify_sequences, batched=True, batch_size=10)